In [5]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

from scipy import stats
from scipy.interpolate import \
    interp2d, griddata, RegularGridInterpolator, RectBivariateSpline, \
    NearestNDInterpolator, LinearNDInterpolator

from vivarium import Artifact # This is causing my kernel to die!!! Memory leak?
import gbd_mapping as gbd
from db_queries import get_ids
from get_draws.api import get_draws

from vivarium_helpers.id_helper import *

# Add path to vivarium_research_lsff to sys.path so imports will work from sibling folders
import sys, os.path
# Note: Assuming pwd = vivarium_research_lsff/nanosim_models
vivarium_research_lsff_path = os.path.abspath("..")
print(vivarium_research_lsff_path)
sys.path.append(vivarium_research_lsff_path)

import lbwsg
import lbwsg_boe # Requires path to vivarium_research_lsff to be in sys.path
import data_processing # Requires path to vivarium_research_lsff to be in sys.path
import lsff_interventions
import demography
import test_lbwsg
import lbwsg_plots
import functions_for_all_nutrients as ffan


# Assumes the path to vivarium_research_lsff is in sys.path
from gbd_data_summary import pull_gbd2019_data as pgd, rank_countries_by_stunting as rbs

!date
!whoami

/ihme/homes/ndbs/vivarium_research_lsff
Fri Apr  9 08:45:26 PDT 2021
ndbs


In [2]:
%load_ext autoreload
%autoreload 2

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
locations = pd.read_csv('../gbd_data_summary/input_data/bmgf_top_25_countries_20201203.csv')
location_ids = locations.location_id.to_list()
locations

,location_name,location_id
0,Angola,168
1,Bangladesh,161
2,Burkina Faso,201
3,Cameroon,202
4,China,6
5,Côte d'Ivoire,205
6,Democratic Republic of the Congo,171
7,Egypt,141
8,Ethiopia,179
9,Ghana,207


In [7]:
%%time
lbwsg_boe.main(
    vivarium_research_lsff_path,
    "/ihme/homes/ndbs/temp/",
    'Nigeria', 10_000, random_seed=3456, num_draws=10, draw_start_idx=0, take_mean=False
)
!ls /ihme/homes/ndbs/temp/

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/tables/atom.py:1224: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return pickle.loads(array.tostring())


flour_fortification_standards_mg_per_kg.csv
gfdx_full_dataset.csv
iron_bw_results_location_id_214.csv
tanzania_art_build.log
CPU times: user 1min 2s, sys: 1.18 s, total: 1min 3s
Wall time: 1min 3s


In [8]:
nigeria_results = pd.read_csv('/ihme/homes/ndbs/temp/iron_bw_results_location_id_214.csv')
print(nigeria_results.shape)
nigeria_results

(48, 15)


/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,location_id,vehicle,coverage_level,measure,age_group_id,draw_43,draw_67,draw_240,draw_298,draw_314,draw_539,draw_558,draw_709,draw_737,draw_767
0,214,maize flour,0.200000,pif,2,0.000392,0.000565,0.000507,0.000361,0.000360,0.000381,0.000351,0.000727,0.000203,0.000218
1,214,maize flour,0.200000,pif,3,0.000296,0.000363,0.000346,0.000234,0.000340,0.000245,0.000229,0.000444,0.000169,0.000217
2,214,maize flour,0.200000,categorical_pif,2,0.000000,0.000024,0.000014,0.000000,0.000021,0.000023,0.000000,0.000210,0.000000,0.000000
3,214,maize flour,0.200000,categorical_pif,3,0.000283,0.000249,0.000267,0.000187,0.000188,0.000180,0.000263,0.000196,0.000202,0.000210
4,214,maize flour,0.200000,averted_dalys,2,5037.809205,8711.949381,7339.205373,5185.912022,4706.867209,4645.763164,5153.574534,9813.625550,3081.683346,2892.999504
5,214,maize flour,0.200000,averted_dalys,3,740.806634,1199.193351,973.256405,752.716244,881.670027,587.585145,617.718406,1284.079097,489.175918,579.833086
6,214,maize flour,0.500000,pif,2,0.000440,0.000615,0.000540,0.000395,0.000429,0.000412,0.000381,0.000780,0.000230,0.000244
7,214,maize flour,0.500000,pif,3,0.000335,0.000418,0.000385,0.000258,0.000382,0.000267,0.000266,0.000606,0.000185,0.000238
8,214,maize flour,0.500000,categorical_pif,2,0.000000,0.000024,0.000026,0.000000,0.000021,0.000023,0.000000,0.000210,0.000000,0.000000
9,214,maize flour,0.500000,categorical_pif,3,0.000288,0.000261,0.000267,0.000187,0.000196,0.000180,0.000266,0.000226,0.000202,0.000210


In [9]:
%%time
lbwsg_boe.main(
    vivarium_research_lsff_path,
    "/ihme/homes/ndbs/temp/",
    'Ethiopia', 10_000, random_seed=3456, num_draws=10, draw_start_idx=0, take_mean=False
)
ethiopia_results = pd.read_csv('/ihme/homes/ndbs/temp/iron_bw_results_location_id_179.csv')
print(ethiopia_results.shape)
ethiopia_results

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/tables/atom.py:1224: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return pickle.loads(array.tostring())


(48, 15)
CPU times: user 1min 1s, sys: 1.04 s, total: 1min 2s
Wall time: 1min 2s


,location_id,vehicle,coverage_level,measure,age_group_id,draw_43,draw_67,draw_240,draw_298,draw_314,draw_539,draw_558,draw_709,draw_737,draw_767
0,179,maize flour,0.200000,pif,2,0.000469,0.002387,0.003734,0.000447,0.000536,0.000660,0.000535,0.000952,0.000176,0.000373
1,179,maize flour,0.200000,pif,3,0.000467,0.002346,0.003715,0.000530,0.000596,0.000716,0.000646,0.001024,0.000225,0.000423
2,179,maize flour,0.200000,categorical_pif,2,0.000189,0.001945,0.000765,0.000638,0.000471,0.000158,0.000201,0.000381,0.000191,0.000015
3,179,maize flour,0.200000,categorical_pif,3,0.000235,0.001600,0.001216,0.000290,0.000669,0.000348,0.000081,0.001022,0.000084,0.000276
4,179,maize flour,0.200000,averted_dalys,2,2020.134646,13648.266437,18434.481019,2345.204470,2297.126574,2721.174588,2748.927155,4668.463101,963.912251,1747.864602
5,179,maize flour,0.200000,averted_dalys,3,505.206891,3271.752821,4773.835178,705.423385,684.002467,760.624828,723.613893,1206.854258,294.155376,476.437064
6,179,maize flour,0.500000,pif,2,0.000574,0.003083,0.004554,0.000574,0.000724,0.000809,0.000699,0.001235,0.000227,0.000469
7,179,maize flour,0.500000,pif,3,0.000580,0.002809,0.004362,0.000622,0.000730,0.000846,0.000785,0.001246,0.000281,0.000490
8,179,maize flour,0.500000,categorical_pif,2,0.000189,0.004547,0.008193,0.000847,0.000710,0.000158,0.000201,0.000407,0.000191,0.000020
9,179,maize flour,0.500000,categorical_pif,3,0.000235,0.001669,0.001302,0.000295,0.001187,0.000348,0.000081,0.001054,0.000084,0.000276


In [14]:
np.random.default_rng(1234).permutation(1000)[10:20]

array([231, 978, 729, 354, 368, 914, 335, 883, 804, 888])

In [15]:
np.random.default_rng(1234).permutation(1000)[0:20]

array([314, 767, 539,  43, 558, 709, 737,  67, 298, 240, 231, 978, 729,
       354, 368, 914, 335, 883, 804, 888])

In [16]:
eval("10")

2021-04-09 00:52:21.139 | INFO     | lbwsg_boe:<module>:3 - pre-imports
2021-04-09 00:52:21.140 | INFO     | lbwsg_boe:<module>:9 - pd, np imported
2021-04-09 00:52:21.141 | INFO     | lbwsg_boe:<module>:15 - made it past imports


10

In [18]:
type(eval('False'))

bool

In [19]:
bool('False')

True

In [20]:
type(eval('10'))

int

In [22]:
# Bytes of memory / (rows of table) = bytes available per row
10e9 / (250*80_000)

400.0

In [24]:
%%time
lbwsg_boe.main(
    vivarium_research_lsff_path,
    "/ihme/homes/ndbs/temp/",
    'India', 10_000, random_seed=3456, num_draws=10, take_mean=False
)
india_results = pd.read_csv('/ihme/homes/ndbs/temp/iron_bw_results_location_id_163.csv')
print(india_results.shape)
india_results

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/tables/atom.py:1224: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return pickle.loads(array.tostring())


(24, 15)
CPU times: user 27.7 s, sys: 510 ms, total: 28.3 s
Wall time: 28.9 s


,location_id,vehicle,coverage_level,measure,age_group_id,draw_43,draw_67,draw_240,draw_298,draw_314,draw_539,draw_558,draw_709,draw_737,draw_767
0,163,wheat flour,0.200000,pif,2,0.000069,0.000065,0.000016,0.000036,0.000084,0.000012,0.000021,0.000160,0.000039,7.556501e-05
1,163,wheat flour,0.200000,pif,3,0.000094,0.000071,0.000043,0.000020,0.000125,0.000016,0.000009,0.000181,0.000039,8.442731e-05
2,163,wheat flour,0.200000,categorical_pif,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
3,163,wheat flour,0.200000,categorical_pif,3,0.000006,0.000000,0.000000,0.000002,0.000005,0.000000,0.000000,0.000001,0.000007,5.309422e-07
4,163,wheat flour,0.200000,averted_dalys,2,2162.974612,2390.284230,476.757285,1277.167574,2298.429415,421.777471,874.717438,5703.669421,1378.278024,2.470555e+03
5,163,wheat flour,0.200000,averted_dalys,3,558.955117,484.134521,242.477321,124.367827,655.142247,98.775603,71.127223,1255.614932,259.770342,5.306132e+02
6,163,wheat flour,0.500000,pif,2,0.000099,0.000088,0.000030,0.000036,0.000119,0.000012,0.000021,0.000186,0.000045,9.226033e-05
7,163,wheat flour,0.500000,pif,3,0.000116,0.000093,0.000050,0.000030,0.000140,0.000034,0.000015,0.000209,0.000048,9.499464e-05
8,163,wheat flour,0.500000,categorical_pif,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
9,163,wheat flour,0.500000,categorical_pif,3,0.000006,0.000000,0.000000,0.000002,0.000139,0.000000,0.000000,0.000086,0.000007,1.266669e-04


In [36]:
%%time
for location in locations.iloc[0:7, 0]:
    print(location)
    lbwsg_boe.main(
    vivarium_research_lsff_path,
    "/ihme/homes/ndbs/temp/",
    location, 100, random_seed=3456, num_draws=10, take_mean=False
    )
!ls /ihme/homes/ndbs/temp/

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Angola


/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/tables/atom.py:1224: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return pickle.loads(array.tostring())


Bangladesh
Burkina Faso


/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/tables/atom.py:1224: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return pickle.loads(array.tostring())


...

China


/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/tables/atom.py:1224: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return pickle.loads(array.tostring())


Côte d'Ivoire
Democratic Republic of the Congo


/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/tables/atom.py:1224: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return pickle.loads(array.tostring())


Angola_parallel.txt
Bangladesh_parallel.txt
Burkina Faso_parallel.txt
Cameroon_parallel.txt
China_parallel.txt
Côte d'Ivoire_parallel.txt
Democratic Republic of the Congo_parallel.txt
flour_fortification_standards_mg_per_kg.csv
gfdx_full_dataset.csv
India_parallel.txt
iron_bw_results__Angola.csv
iron_bw_results__Bangladesh.csv
iron_bw_results__Burkina Faso.csv
iron_bw_results__Cameroon.csv
iron_bw_results__China.csv
iron_bw_results__Côte d'Ivoire.csv
iron_bw_results__Democratic Republic of the Congo.csv
iron_bw_results_location_id_163.csv
iron_bw_results_location_id_179.csv
iron_bw_results_location_id_214.csv
tanzania_art_build.log
CPU times: user 13 s, sys: 1.56 s, total: 14.5 s
Wall time: 16.1 s


In [38]:
!head /ihme/homes/ndbs/temp/iron_bw_results__Côte\ d\'Ivoire.csv

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


location_id,vehicle,coverage_level,measure,age_group_id,draw_43,draw_67,draw_240,draw_298,draw_314,draw_539,draw_558,draw_709,draw_737,draw_767
205,maize flour,0.2,pif,2,5.141048458909828e-05,0.00025411040592927044,0.0001492870413904061,1.499197636025876e-06,0.0001140590597253582,5.042464938155675e-05,5.9628933447424525e-05,4.104182475402589e-05,0.0,3.163148895563676e-05
205,maize flour,0.2,pif,3,0.0,9.249753521270991e-06,0.0,0.0,2.5408045205056352e-06,0.0,1.67065061242134e-06,0.0,0.0,0.0
205,maize flour,0.2,categorical_pif,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205,maize flour,0.2,categorical_pif,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205,maize flour,0.2,averted_dalys,2,59.439328810701525,354.5025460440987,162.79663634227038,2.0650977197895832,117.08010410717297,58.73306981377165,83.78509634568032,48.6757639042173,0.0,32.39668830558643
205,maize flour,0.2,averted_dalys,3,0.0,3.2641290211424736,0.0,0.0,0.7002588589752917,0.0,0.39382635024209667,0.0,0.0,0.0
205,maize flour,0.5,pif

In [1]:
250*80_000

20000000

In [2]:
250*2*2

1000

In [3]:
250*80_000/(250*2*2)

20000.0

In [6]:
!ls results2

Egypt_parallel.txt	       iron_bw_results__Pakistan.csv
Ethiopia_parallel.txt	       iron_bw_results__South Africa.csv
Ghana_parallel.txt	       Kenya_parallel.txt
Indonesia_parallel.txt	       Nigeria_parallel.txt
iron_bw_results__Ethiopia.csv  Pakistan_parallel.txt
iron_bw_results__Ghana.csv     South Africa_parallel.txt
iron_bw_results__Kenya.csv     Sudan_parallel.txt
iron_bw_results__Nigeria.csv


/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
pd.read_csv('results2/iron_bw_results__Ethiopia.csv')

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,location_id,vehicle,coverage_level,measure,age_group_id,draw_4,draw_6,draw_10,draw_13,draw_16,...,draw_971,draw_973,draw_974,draw_978,draw_981,draw_985,draw_988,draw_992,draw_994,draw_999
0,179,maize flour,0.200000,pif,2,0.000397,0.001212,0.002131,0.000205,0.001063,...,0.000284,0.000392,0.000854,0.000770,0.000646,0.000401,0.000613,0.000375,0.000465,0.000233
1,179,maize flour,0.200000,pif,3,0.000410,0.001294,0.002172,0.000233,0.001098,...,0.000285,0.000431,0.000880,0.000857,0.000687,0.000441,0.000697,0.000375,0.000495,0.000254
2,179,maize flour,0.200000,categorical_pif,2,0.000015,0.000210,0.000606,0.000027,0.000507,...,0.000033,0.000027,0.000199,0.000280,0.000287,0.000052,0.000248,0.000031,0.000076,0.000041
3,179,maize flour,0.200000,categorical_pif,3,0.000237,0.000645,0.001202,0.000023,0.000461,...,0.000178,0.000311,0.000374,0.000734,0.000326,0.000132,0.000203,0.000066,0.000257,0.000020
4,179,maize flour,0.200000,averted_dalys,2,2172.392734,6126.587565,10813.974916,1233.560070,4066.998080,...,1614.616416,1960.428192,5384.791229,4174.294710,3227.330599,1843.232042,2809.966424,1738.276438,1820.343391,1069.195489
5,179,maize flour,0.200000,averted_dalys,3,561.537457,1396.095702,2698.892535,314.081974,1020.064281,...,384.894671,539.237892,1383.316192,1122.250067,836.227322,494.296418,739.717489,421.244126,484.889283,255.856648
6,179,maize flour,0.500000,pif,2,0.000481,0.001529,0.002608,0.000262,0.001340,...,0.000364,0.000488,0.001062,0.000951,0.000798,0.000503,0.000768,0.000458,0.000578,0.000299
7,179,maize flour,0.500000,pif,3,0.000509,0.001578,0.002661,0.000282,0.001351,...,0.000346,0.000526,0.001089,0.001070,0.000852,0.000552,0.000904,0.000467,0.000618,0.000310
8,179,maize flour,0.500000,categorical_pif,2,0.000027,0.000505,0.000646,0.000026,0.000753,...,0.000064,0.000030,0.000447,0.000307,0.000321,0.000054,0.000254,0.000054,0.000120,0.000039
9,179,maize flour,0.500000,categorical_pif,3,0.000321,0.000738,0.001407,0.000034,0.000767,...,0.000188,0.000353,0.000502,0.000791,0.000359,0.000217,0.000218,0.000140,0.000357,0.000032


In [8]:
gday = pd.read_csv("../data_prep/outputs/lsff_gday_data.csv")
gday

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,location_id,location_name,sub_population,vehicle,metric,mass_unit,time_unit,pop_denom,value_mean,lower,upper
0,163,India,total population,wheat flour,mean,g,day,capita,169.620000,113.134860,226.105140
1,179,Ethiopia,total population,maize flour,mean,g,day,capita,113.840000,68.762191,158.917809
2,179,Ethiopia,total population,wheat flour,mean,g,day,capita,97.640000,65.124913,130.155087
3,214,Nigeria,total population,maize flour,mean,g,day,capita,95.100000,57.442765,132.757235
4,214,Nigeria,total population,wheat flour,mean,g,day,capita,49.150000,32.782563,65.517437
5,161,Bangladesh,women of reproductive age,wheat flour,mean,g,day,capita,77.900000,51.958528,103.841472
6,165,Pakistan,women of reproductive age,wheat flour,mean,g,day,capita,86.175260,57.478045,114.872475
7,189,United Republic of Tanzania,total population,maize flour,mean,g,day,capita,172.470000,102.944854,241.995146
8,189,United Republic of Tanzania,total population,wheat flour,mean,g,day,capita,32.960000,21.983994,43.936006
9,190,Uganda,total population,maize flour,mean,g,day,capita,126.300000,75.386647,177.213353


In [9]:
gday.vehicle.unique()

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['wheat flour', 'maize flour'], dtype=object)